In [1]:
using CUDA, Random
using GPUArrays: @allowscalar
using Distributions: Gamma
CUDA.allowscalar(false)
include("utils.jl")
include("statistics.jl")

_var (generic function with 2 methods)

In [2]:
# permutation test configuration
pooled = true
alpha = 0.05
alternative = "two_sided"

"two_sided"

In [3]:
# data dimensions
B  = 100   # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 6      # size of group 1
ny = 6      # size of group 2
px, py = partition(nx, ny)
dtype = Float32

Float32

In [4]:
# Generate data
Random.seed!(123)

shapeX = rand(1:10, B)
scaleX = rand(1:10, B)
distrX = Gamma.(shapeX, scaleX)

shapeY = rand(1:10, B)
scaleY = rand(1:10, B)
distrY = Gamma.(shapeY, scaleY)

deltas = @. (shapeX * scaleX) - (shapeY * scaleY)

# produces B vectors of S * nx elements
# each vector is drawn from a distinct (but not necessarily distinct) distribution
x = rand.(distrX, S * nx)
y = rand.(distrY, S * ny)

# flatten into (B, S * n)
x = hcat(x...)'
y = hcat(y...)'

# reshape to 3D batch
x = reshape(x, (B, S, nx))
y = reshape(y, (B, S, ny))

# Compute t confidence intervals for each of the (B x S x nx) pairs
wide   = tconf(x, y, alpha=0.0001, pooled=pooled)
narrow = tconf(x, y, alpha=0.3, pooled=pooled)

wide   = reshape(wide, B, S)
narrow = reshape(narrow, B, S)

# move data to GPU
x = CuArray(x)
y = CuArray(y)
typeof(x)

CuArray{Float64, 3, CUDA.Mem.DeviceBuffer}

In [5]:
function pval(x, y, pooled=false, alternative="two-sided", delta=0)
    """
    Parameters
    ----------
    x : Vector{Real}
        Data for group 1
    
    y : Vector{Real}
        Data for group 2
    
    pooled : Bool
        Assume equal/unequal variances for the two groups
    
    alternative : String
        Type of alternative hypothesis
    
    delta : Real
        Null hypothesis difference in means
    
    Returns
    -------
    Float64
        Proportion of pairs among all sample combinations which have
        a test statistic as or more extreme than the original pair (x, y)
    """
    x_shift = x .- delta  # shift group 1 under null hypothesis
    t_obs = t(x_shift, y, pooled)  # test statistic for observed data
    combined = vcat(x_shift, y)  # join original pair into single vector
    xs = combined[px]   # get all combinations of pairs from original pair
    ys = combined[py]
    ts = t(xs, ys, pooled)   # test statistic for all possible pairs of samples
    
    if alternative == "smaller"
        n_extreme = count(ts .<= t_obs)
    elseif alternative == "larger"
        n_extreme = count(ts .>= t_obs)
    else
        n_extreme = count(@. (ts <= -abs(t_obs)) | (ts >= abs(t_obs)))
    end

    return n_extreme / size(px, 1)  # proportion of pairs w/ extreme test statistic
    
end

function search(x, y, start, stop;
                pooled=false, alternative="two-sided", isLowerBound=true,
                margin=0.005, threshold=1.0, alpha=0.05)
    p_start = pval(x, y, pooled, alternative, start)
    p_end   = pval(x, y, pooled, alternative, stop)
    #println("p_start = ", p_start, ", p_end = ", p_end)
    
    # p-values corresponding to `start` and `stop` must be on opposite sides of `alpha`
    @assert (p_start - alpha) * (p_end - alpha) <= 0

    p = p_new = delta = nothing
    percent_change = (old, new) -> 100 * abs(new-old) / old
    
    while true
        delta = (start + stop) / 2
        p_new = pval(x, y, pooled, alternative, delta)

        if !isnothing(p) && percent_change(p, p_new) <= threshold
            break  # (1) percent change in p-value is below `threshold`
        end
        
        compare = (alpha - p_new) - isLowerBound * 2 * (alpha - p_new)
        if margin < compare
            stop = delta
        elseif margin < -compare
            start = delta
        else
            break  # (2) p-value is within `margin` of `alpha`
        end

        p = p_new
    end
    
    return delta
end

function permInterval(x, y, wide, narrow, delta_true; pooled=false, alpha=0.05, alternative="two_sided")
    """Returns true (false) if permutation test confidence interval does (not) include difference in
    population means.
    Parameters
    ----------
    x1 : Vector{Float64}
        Data for group 1
    x2 : Vector{Float64}
        Data for group 2
    partitions : Tuple{Matrix{Int64}, Matrix{Int64}}
        The i-th rows of x1[partitions[1]] and x2[partitions[2]] denote the i-th arrangement of
        the original (n1+n2) observations into two groups of size n1 and n2.
    delta_true : Float64
        Difference in population means
    pooled : Bool
        Assume pooled or unpooled variances
    alpha : Float64
        Significance level
    alternative : String
        Type of alternative hypothesis ("two-sided", "smaller", "larger")
    Returns
    -------
    Bool
        True (false) if permutation test confidence interval does (not) include difference in population means.
    """

    # use binary search to find approximate permutation test confidence interval

    lo = search(x, y, wide[1], narrow[1],
                pooled=pooled, alpha=alpha, alternative=alternative, isLowerBound=true)
    hi = search(x, y, narrow[2], wide[2],
                pooled=pooled, alpha=alpha, alternative=alternative, isLowerBound=false)
    # println("(", lo, ", ", hi, ")")
    return lo <= delta_true <= hi
end

function coverage(xs, ys, wide, narrow, delta_true; pooled=false, alpha=0.05, alternative="two_sided")
    covered = permInterval.(eachrow(xs), eachrow(ys), wide, narrow, delta_true,
                            pooled=pooled, alpha=alpha, alternative=alternative)
    return count(covered) / S
end

coverage (generic function with 1 method)

In [6]:
@time begin
results = coverage.(eachslice(x, dims=1),
                    eachslice(y, dims=1),
                    eachrow(wide),
                    eachrow(narrow),
                    deltas,
                    pooled=pooled,
                    alpha=alpha,
                    alternative=alternative)
end

outfile = "results_temp.txt"
open(outfile, "w") do file
    println(
        file,
        "==============================================\n",
        "num. batches : ", B, "\n",
        "batch size   : ", S, "\n",
        "size group 1 : ", nx, "\n",
        "size group 2 : ", ny, "\n",
        "alternative  : ", alternative, "\n",
        "pooled       : ", pooled, "\n",
        "alpha        : ", alpha, "\n",
        "=============================================="
    )
    for (i, coverageProb) in enumerate(results)
        println(file, coverageProb, " ; x: ", distrX[i], " ; y: ", distrY[i])
    end
end

2132.490683 seconds (3.46 G allocations: 189.464 GiB, 4.71% gc time, 1.72% compilation time)


- overview/explain two-sample t-test
- derive t test confidence interval by inverting the t test of H0: delta = delta0 vs. H_A: delta != delta0, where delta = mu_1 - mu_2, delta0 in R
    - CI is the set of all deltas for which we do NOT reject the null == set of all deltas for which p-value > 0.05
    - set of all deltas for which the test statistic < t quantile == set of all deltas for which [mean(x1) - mean(x2) - delta] / s.d. (pooled or unpooled) < t quantile
    - output after solving for delta is the CI
- mention that the test statistic for difference in means (for pooled variances) follows a t distribution
    - DoF also changes for unpooled variances
- permutation test
    - explain how permTest p-value is derived (detail interval halving, etc.)
    - similar to above t test overview, start with two-sided alternative, then explain for inverting two one-sided tests (here, use alpha/2 instead of alpha)
    - literature review
- fix DoF in unpooled tconf()